In [1]:
# Célula 1: Instalação de Bibliotecas
!pip install google-generativeai qdrant-client python-dotenv pandas pypdf gradio


In [1]:
#CÉLULA 2 - Importações e configuração do Gemini e do Qdrant
import os
import time
import pandas as pd # Para carregar dados de exemplo, opcional
from dotenv import load_dotenv
import google.generativeai as genai
from qdrant_client import QdrantClient, models

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Configurar a API do Google AI (Gemini)
GOOGLE_AI_API_KEY = os.getenv("GOOGLE_AI_API_KEY")
if not GOOGLE_AI_API_KEY:
    print("AVISO: GOOGLE_AI_API_KEY não encontrada. Verifique seu arquivo .env ou defina a variável.")
else:
    genai.configure(api_key=GOOGLE_AI_API_KEY)
    print("SDK do Google AI (Gemini) configurado com sucesso.")

# Configurar o cliente Qdrant
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

qdrant_client = None # Inicializa como None

if QDRANT_URL and "cloud.qdrant.io" in QDRANT_URL: # Verifica se parece uma URL de nuvem
    print(f"Tentando conectar ao Qdrant Cloud em: {QDRANT_URL}")
    if not QDRANT_API_KEY:
        print("AVISO: QDRANT_API_KEY não encontrada para Qdrant Cloud. A conexão pode falhar.")
    try:
        qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY, timeout=60) # Timeout aumentado
        qdrant_client.get_collections() # Testa a conexão
        print("Conectado ao Qdrant Cloud com sucesso!")
    except Exception as e:
        print(f"ERRO ao conectar ao Qdrant Cloud: {e}")
        print("Fallback para Qdrant rodando localmente ou em memória.")
        qdrant_client = None # Garante que não usaremos uma instância falha

if not qdrant_client: # Se não conectou à nuvem ou não havia URL da nuvem
    try:
        # Tenta conectar a uma instância local do Qdrant
        qdrant_client = QdrantClient(host="localhost", port=6333)
        qdrant_client.get_collections() # Testa a conexão
        print("Conectado ao Qdrant local (localhost:6333) com sucesso!")
    except Exception as e_local:
        print(f"ERRO ao conectar ao Qdrant local: {e_local}")
        qdrant_client = QdrantClient(":memory:")
        print("ATENÇÃO: Qdrant rodando em memória. Os dados serão perdidos ao fechar.")

print("\n--- Configurações de clientes concluídas. ---")

SDK do Google AI (Gemini) configurado com sucesso.
Tentando conectar ao Qdrant Cloud em: https://1346b74e-7bf3-489a-b4cc-b1eacb715c46.europe-west3-0.gcp.cloud.qdrant.io
Conectado ao Qdrant Cloud com sucesso!

--- Configurações de clientes concluídas. ---


In [3]:
# CÉLULA 3 - Criação da coleção no qdrant

# Definições para a coleção e embeddings
COLLECTION_NAME = "fisica_ensino_medio_chunks" # Nome da coleção
GEMINI_EMBEDDING_MODEL = 'models/embedding-001' # Modelo de embedding do Gemini
GEMINI_EMBEDDING_DIMENSION = 768  # Dimensão dos embeddings para 'models/embedding-001'
MODELO_GEMINI_PARA_GERACAO = "gemini-1.5-flash-latest" # Ou "gemini-1.0-pro-latest", etc.

if qdrant_client: # Assume que qdrant_client foi inicializado na Célula 2
    try:
        collection_info = qdrant_client.get_collection(collection_name=COLLECTION_NAME)
        print(f"Coleção '{COLLECTION_NAME}' já existe.")
        
        current_dimension = None
        vectors_config = collection_info.config.params.vectors

        # Para coleções criadas com models.VectorParams (vetor único padrão), 
        # vectors_config será uma instância de models.VectorParams.
        if isinstance(vectors_config, models.VectorParams):
            current_dimension = vectors_config.size
        elif isinstance(vectors_config, dict) and models.DEFAULT_VECTOR_NAME in vectors_config:
             # Caso de vetores nomeados, pegamos o default se existir
            current_dimension = vectors_config[models.DEFAULT_VECTOR_NAME].size
        else:
            print(f"AVISO: Não foi possível determinar a configuração de vetores da coleção '{COLLECTION_NAME}' da forma esperada.")
            # Você pode adicionar mais lógica aqui se usar configurações de vetores mais complexas.

        if current_dimension is not None: # Prossegue apenas se a dimensão foi encontrada
            if current_dimension != GEMINI_EMBEDDING_DIMENSION:
                print(f"ALERTA: Dimensão da coleção existente ({current_dimension}) é DIFERENTE da esperada ({GEMINI_EMBEDDING_DIMENSION}).")
                print("Isso CAUSARÁ ERROS ao tentar inserir ou buscar vetores com a dimensão incorreta.")
                print(f"Soluções: Use outra coleção, apague e recrie esta coleção '{COLLECTION_NAME}', ou ajuste GEMINI_EMBEDDING_DIMENSION se o modelo mudou.")
                # Exemplo para apagar (CUIDADO: APAGA TODOS OS DADOS DA COLEÇÃO):
                # print(f"Para apagar e recriar, descomente e execute: qdrant_client.delete_collection(collection_name=COLLECTION_NAME)")
                # raise Exception("Dimensão da coleção incompatível. Interrompendo para evitar mais erros.")
            else:
                print(f"Dimensão da coleção existente ({current_dimension}) é compatível com a dimensão esperada ({GEMINI_EMBEDDING_DIMENSION}).")
        elif collection_info: # Se current_dimension não foi setada mas a coleção existe
             print(f"Não foi possível verificar a dimensão da coleção '{COLLECTION_NAME}'. Prossiga com cautela.")


    except Exception as e:
        # Trata erros como "não encontrado" de forma mais genérica
        # Também inclui o caso de ValueError que pode vir de um `raise Exception` acima se descomentado.
        if "not found" in str(e).lower() or "status_code=404" in str(e).lower() or "status code 404" in str(e).lower() or isinstance(e, ValueError):
            print(f"Coleção '{COLLECTION_NAME}' não encontrada ou marcada para recriação. Criando nova coleção...")
            try:
                qdrant_client.create_collection(
                    collection_name=COLLECTION_NAME,
                    vectors_config=models.VectorParams(
                        size=GEMINI_EMBEDDING_DIMENSION,
                        distance=models.Distance.COSINE
                    )
                )
                print(f"Coleção '{COLLECTION_NAME}' criada com sucesso com dimensão {GEMINI_EMBEDDING_DIMENSION}.")
            except Exception as e_create:
                print(f"ERRO CRÍTICO ao criar a coleção '{COLLECTION_NAME}': {e_create}")
        else:
            print(f"Erro desconhecido ao verificar/criar coleção '{COLLECTION_NAME}': {e}")
            import traceback
            traceback.print_exc()
else:
    print("ERRO CRÍTICO: Cliente Qdrant não inicializado na Célula 2. Não é possível configurar a coleção.")

print("\n--- Configuração da coleção Qdrant concluída. ---")

Coleção 'fisica_ensino_medio_chunks' já existe.
Dimensão da coleção existente (768) é compatível com a dimensão esperada (768).

--- Configuração da coleção Qdrant concluída. ---


In [5]:
# CÉLULA 4 - Definição da função gerar_e_armazenar_embeddings
import uuid # Para IDs únicos, se necessário

def gerar_e_armazenar_embeddings_gemini(text_chunks_dicts, collection_name, q_client, gemini_model,
                                        qdrant_upsert_batch_size=100, gemini_api_call_batch_size=30):
    """
    Gera embeddings para uma lista de chunks (dicionários) usando Gemini e os armazena no Qdrant.
    Cada chunk é um dicionário esperado com, no mínimo, a chave "texto_original".
    """
    if not GOOGLE_AI_API_KEY:
        print("ERRO: Chave da API do Google AI não configurada. Abortando.")
        return
    if not q_client:
        print("ERRO: Cliente Qdrant não configurado. Abortando.")
        return
    if not text_chunks_dicts:
        print("AVISO: Lista de text_chunks_dicts está vazia. Nada para processar.")
        return

    pontos_para_upsert = []
    total_chunks_count = len(text_chunks_dicts)
    chunks_processados_global = 0
    
    # Usaremos um contador global para IDs sequenciais simples nesta execução.
    # Para robustez entre múltiplas execuções ou atualizações incrementais, UUIDs são melhores.
    id_counter = 0 

    print(f"Iniciando geração e armazenamento de embeddings com Gemini para {total_chunks_count} chunks...")
    start_time_total_process = time.time()
    last_log_time = time.time()
    log_interval_seconds = 10

    for i in range(0, total_chunks_count, gemini_api_call_batch_size):
        current_batch_chunk_dicts = text_chunks_dicts[i:i + gemini_api_call_batch_size]
        if not current_batch_chunk_dicts:
            continue

        texts_for_embedding_api = []
        valid_dicts_in_batch = [] # Para manter correspondência com os embeddings retornados
        for chunk_dict in current_batch_chunk_dicts:
            if isinstance(chunk_dict, dict) and "texto_original" in chunk_dict and chunk_dict["texto_original"].strip():
                texts_for_embedding_api.append(chunk_dict["texto_original"])
                valid_dicts_in_batch.append(chunk_dict)
            else:
                print(f"  AVISO: Chunk inválido ou sem texto_original encontrado no lote (índice original ~{i+len(valid_dicts_in_batch)}), será pulado para embedding.")
        
        if not texts_for_embedding_api:
            print(f"  AVISO: Nenhum texto válido para embedding no lote iniciando em {i}. Pulando lote da API Gemini.")
            chunks_processados_global += len(current_batch_chunk_dicts) # Conta como processados para o loop principal
            continue
            
        print(f"\nProcessando lote de {len(texts_for_embedding_api)} textos válidos para API Gemini (de {len(current_batch_chunk_dicts)} chunks originais no lote)...")
        
        try:
            result = genai.embed_content(
                model=gemini_model,
                content=texts_for_embedding_api,
                task_type="RETRIEVAL_DOCUMENT"
            )
            batch_embeddings_vectors = result['embedding']

            if len(batch_embeddings_vectors) != len(texts_for_embedding_api):
                print(f"  ALERTA: Discrepância no lote Gemini. Textos enviados: {len(texts_for_embedding_api)}, Embeddings recebidos: {len(batch_embeddings_vectors)}. Pulando este lote da API.")
                chunks_processados_global += len(current_batch_chunk_dicts)
                continue

            for original_chunk_dict, vector_embedding in zip(valid_dicts_in_batch, batch_embeddings_vectors):
                payload_content = original_chunk_dict.copy()
                
                # Geração de ID:
                # Se você limpar a coleção a cada vez, um contador simples pode funcionar.
                # Se for adicionar incrementalmente, use UUIDs ou um hash do conteúdo + origem.
                # current_id = str(uuid.uuid4()) # Opção UUID
                current_id = id_counter # Opção sequencial simples para esta execução
                id_counter += 1

                pontos_para_upsert.append(
                    models.PointStruct(
                        id=current_id,
                        vector=vector_embedding,
                        payload=payload_content
                    )
                )
            
            print(f"  Lote da API Gemini processado. {len(batch_embeddings_vectors)} embeddings gerados.")

        except Exception as e:
            print(f"  ERRO CRÍTICO ao processar lote da API Gemini (iniciando em chunk {i}): {e}")
            print(f"  Este lote será pulado. Tentando continuar com o próximo lote...")
            import traceback
            traceback.print_exc()
            time.sleep(5) # Pausa maior em caso de erro de API
            # Importante: incrementar chunks_processados_global mesmo em erro para o loop avançar
            # ou ter uma lógica de retentativa mais sofisticada.
            # Por simplicidade, aqui estamos pulando o lote inteiro em caso de erro na API.
        
        chunks_processados_global += len(current_batch_chunk_dicts) # Atualiza com base no lote original processado/tentado

        # Upsert no Qdrant
        if len(pontos_para_upsert) >= qdrant_upsert_batch_size or \
           (chunks_processados_global >= total_chunks_count and len(pontos_para_upsert) > 0): # >= para o caso de pularmos lotes
            if pontos_para_upsert: # Apenas se houver pontos para enviar
                print(f"  Enviando {len(pontos_para_upsert)} pontos para Qdrant...")
                try:
                    q_client.upsert(
                        collection_name=collection_name,
                        points=pontos_para_upsert,
                        wait=True
                    )
                    print(f"  {len(pontos_para_upsert)} pontos enviados ao Qdrant.")
                except Exception as e_qdrant:
                    print(f"  ERRO CRÍTICO ao enviar pontos para o Qdrant: {e_qdrant}")
                    traceback.print_exc()
                finally:
                    pontos_para_upsert = [] # Limpa mesmo em erro para não reenviar os mesmos pontos
        
        # Delay entre chamadas à API Gemini
        if i + gemini_api_call_batch_size < total_chunks_count: # Evita sleep após o último lote
             print(f"  Aguardando para a próxima chamada à API Gemini...")
             time.sleep(1.1) # Pausa para respeitar limites de RPM (60 RPM para embedding-001 free tier)


        current_time = time.time()
        if current_time - last_log_time > log_interval_seconds or chunks_processados_global >= total_chunks_count:
            progress_percent = (chunks_processados_global / total_chunks_count) * 100 if total_chunks_count > 0 else 0
            elapsed_total = current_time - start_time_total_process
            print(f"\nProgresso Atual: {progress_percent:.2f}% ({chunks_processados_global}/{total_chunks_count} chunks avaliados). Tempo total: {elapsed_total:.2f}s")
            last_log_time = current_time

    # Garantir que qualquer ponto restante seja enviado
    if len(pontos_para_upsert) > 0 and q_client:
        print(f"\nEnviando lote final de {len(pontos_para_upsert)} pontos para Qdrant...")
        try:
            q_client.upsert(
                collection_name=collection_name,
                points=pontos_para_upsert,
                wait=True
            )
            print(f"  {len(pontos_para_upsert)} pontos finais enviados ao Qdrant.")
        except Exception as e_qdrant_final:
            print(f"  ERRO CRÍTICO ao enviar lote final para o Qdrant: {e_qdrant_final}")
            import traceback
            traceback.print_exc()
        finally:
            pontos_para_upsert = []


    print(f"\n--- Geração e armazenamento de embeddings com Gemini CONCLUÍDOS! Total de {chunks_processados_global} chunks avaliados. ---")

In [7]:
# CÉLULA 5 - Leitura dos PDFs + divisão em chunks
# Esta célula é, portanto, a etapa de PRÉ-PROCESSAMENTO de dados que alimenta o 
# restante do pipeline de IA do notebook. Ela transforma o conteúdo bruto dos PDFs 
# em um formato ESTRUTURADO (lista de dicionários de chunks) pronto para a vetorização.
from pypdf import PdfReader 
import os # Para pegar o nome do arquivo

def extrair_texto_pdf(caminho_pdf):
    """Extrai texto de um arquivo PDF, uma string por página, incluindo o nome do arquivo no log."""
    textos_das_paginas = []
    nome_arquivo = os.path.basename(caminho_pdf) # Pega apenas o nome do arquivo do caminho
    try:
        leitor = PdfReader(caminho_pdf)
        num_paginas = len(leitor.pages)
        print(f"Lendo PDF: '{nome_arquivo}' com {num_paginas} páginas.")
        for i, pagina in enumerate(leitor.pages):
            texto_pagina = pagina.extract_text()
            if texto_pagina:
                # Adiciona metadados da página ao texto do chunk (ainda como string temporariamente)
                textos_das_paginas.append({"texto_bruto": texto_pagina.strip(), "pagina_num": i + 1, "arquivo_origem": nome_arquivo})
            else:
                print(f"  Aviso: Não foi possível extrair texto da página {i+1} do arquivo '{nome_arquivo}'.")
        print(f"Texto extraído de {len(textos_das_paginas)} páginas do arquivo '{nome_arquivo}'.")
    except FileNotFoundError:
        print(f"ERRO: Arquivo PDF não encontrado em '{caminho_pdf}'.")
        return []
    except Exception as e:
        print(f"ERRO ao ler o PDF '{caminho_pdf}': {e}")
        return []
    return textos_das_paginas

def dividir_texto_em_chunks_com_metadata(textos_com_metadata_pagina, max_chars_por_chunk=2000, overlap_chars=200):
    """Divide textos (que já são dicionários com metadados de página) em chunks menores."""
    chunks_finais_formatados = []
    for item_pagina in textos_com_metadata_pagina:
        texto_original_pagina = item_pagina["texto_bruto"]
        pagina_num = item_pagina["pagina_num"]
        arquivo_origem = item_pagina["arquivo_origem"]

        if len(texto_original_pagina) <= max_chars_por_chunk:
            chunks_finais_formatados.append({
                "texto_original": texto_original_pagina,
                "pagina": pagina_num,
                "arquivo_origem": arquivo_origem
            })
            continue
        
        print(f"Dividindo texto da página {pagina_num} do arquivo '{arquivo_origem}' (aprox {len(texto_original_pagina)} caracteres)...")
        
        inicio = 0
        while inicio < len(texto_original_pagina):
            fim = min(inicio + max_chars_por_chunk, len(texto_original_pagina))
            chunk_texto = texto_original_pagina[inicio:fim]
            
            chunks_finais_formatados.append({
                "texto_original": chunk_texto,
                "pagina": pagina_num, # A página original de onde este chunk veio
                "arquivo_origem": arquivo_origem
            })
            
            if fim == len(texto_original_pagina):
                break 
            
            inicio += (max_chars_por_chunk - overlap_chars)
            if inicio >= len(texto_original_pagina):
                 break
    # print(f"Total de chunks após divisão para este PDF: {len(chunks_finais_formatados)}") # Removido para log global
    return chunks_finais_formatados


# Lista dos caminhos para os os arquivos PDF
LISTA_CAMINHOS_PDFS = [
    r"G:\Meu Drive\Acadêmico\CC - Unipê\P6 - 2025.1\Fundamentos_IA\Trabalho_AV2_RAG\Apostila-Física.pdf",
    r"G:\Meu Drive\Acadêmico\CC - Unipê\P6 - 2025.1\Fundamentos_IA\Trabalho_AV2_RAG\fisica2.pdf",
    r"G:\Meu Drive\Acadêmico\CC - Unipê\P6 - 2025.1\Fundamentos_IA\Trabalho_AV2_RAG\fisica3.pdf" ]

todos_os_chunks_formatados = [] # Lista global para todos os chunks de todos os PDFs

for caminho_pdf_atual in LISTA_CAMINHOS_PDFS:
    print(f"\n--- Processando arquivo: {os.path.basename(caminho_pdf_atual)} ---")
    # Extrai o texto bruto, já como lista de dicionários por página
    textos_por_pagina_com_meta = extrair_texto_pdf(caminho_pdf_atual)

    if textos_por_pagina_com_meta:
        # Divide os textos das páginas em chunks menores, mantendo/adicionando metadados
        chunks_do_pdf_atual = dividir_texto_em_chunks_com_metadata(
            textos_por_pagina_com_meta,
            max_chars_por_chunk=2000,
            overlap_chars=200
        )
        todos_os_chunks_formatados.extend(chunks_do_pdf_atual)
        print(f"Adicionados {len(chunks_do_pdf_atual)} chunks do arquivo '{os.path.basename(caminho_pdf_atual)}'.")

# A variável final que será usada nas próximas células
chunks_de_texto = todos_os_chunks_formatados

if chunks_de_texto:
    print(f"\nTotal de {len(chunks_de_texto)} chunks de texto preparados de {len(LISTA_CAMINHOS_PDFS)} arquivo(s) PDF.")
    print("\nExemplo do primeiro chunk formatado:")
    if isinstance(chunks_de_texto[0], dict):
        for chave, valor in chunks_de_texto[0].items():
            print(f"  {chave}: {str(valor)[:200]}...") 
    else:
        print(str(chunks_de_texto[0])[:500] + "...")
else:
    print("\nAVISO: Nenhum chunk de texto foi preparado. Verifique os caminhos dos PDFs e a extração.")

print("\n--- Preparação dos dados de múltiplos PDFs concluída. ---")


--- Processando arquivo: Apostila-Física.pdf ---
Lendo PDF: 'Apostila-Física.pdf' com 48 páginas.
Texto extraído de 48 páginas do arquivo 'Apostila-Física.pdf'.
Dividindo texto da página 2 do arquivo 'Apostila-Física.pdf' (aprox 3319 caracteres)...
Dividindo texto da página 3 do arquivo 'Apostila-Física.pdf' (aprox 3794 caracteres)...
Dividindo texto da página 4 do arquivo 'Apostila-Física.pdf' (aprox 2649 caracteres)...
Dividindo texto da página 6 do arquivo 'Apostila-Física.pdf' (aprox 2463 caracteres)...
Dividindo texto da página 7 do arquivo 'Apostila-Física.pdf' (aprox 2156 caracteres)...
Dividindo texto da página 10 do arquivo 'Apostila-Física.pdf' (aprox 2283 caracteres)...
Dividindo texto da página 15 do arquivo 'Apostila-Física.pdf' (aprox 2581 caracteres)...
Dividindo texto da página 16 do arquivo 'Apostila-Física.pdf' (aprox 2660 caracteres)...
Dividindo texto da página 18 do arquivo 'Apostila-Física.pdf' (aprox 2212 caracteres)...
Dividindo texto da página 19 do arquivo 'A

In [9]:
# CÉLULA 6 -- Chamada da função gerar_e_armazenar_embeddings ==> Geração e armazenamento de embeddings no banco de dados vetorial → qdrant

if qdrant_client and chunks_de_texto and GOOGLE_AI_API_KEY:
    gerar_e_armazenar_embeddings_gemini(
        text_chunks_dicts=chunks_de_texto,  
        collection_name=COLLECTION_NAME,
        q_client=qdrant_client,
        gemini_model=GEMINI_EMBEDDING_MODEL
        # Os argumentos opcionais qdrant_upsert_batch_size e gemini_api_call_batch_size 
        # usarão seus valores padrão definidos na função, se não forem especificados aqui.
    )
elif not qdrant_client:
    print("Execução abortada: Cliente Qdrant não está inicializado.")
elif not GOOGLE_AI_API_KEY: # Certifique-se que GOOGLE_AI_API_KEY está definida
    print("Execução abortada: Chave da API do Google AI não configurada.")
elif not chunks_de_texto: # Adicionada verificação explícita para clareza
    print("Execução abortada: `chunks_de_texto` estão vazios ou não definidos.")
else: # Caso genérico, embora as condições acima devam cobrir os principais
    print("Execução abortada por motivo desconhecido ou condição não atendida.")

Iniciando geração e armazenamento de embeddings com Gemini para 1264 chunks...

Processando lote de 30 textos válidos para API Gemini (de 30 chunks originais no lote)...
  Lote da API Gemini processado. 30 embeddings gerados.
  Aguardando para a próxima chamada à API Gemini...

Processando lote de 30 textos válidos para API Gemini (de 30 chunks originais no lote)...
  Lote da API Gemini processado. 30 embeddings gerados.
  Aguardando para a próxima chamada à API Gemini...

Processando lote de 30 textos válidos para API Gemini (de 30 chunks originais no lote)...
  Lote da API Gemini processado. 30 embeddings gerados.
  Aguardando para a próxima chamada à API Gemini...

Processando lote de 30 textos válidos para API Gemini (de 30 chunks originais no lote)...
  Lote da API Gemini processado. 30 embeddings gerados.
  Enviando 120 pontos para Qdrant...
  120 pontos enviados ao Qdrant.
  Aguardando para a próxima chamada à API Gemini...

Progresso Atual: 9.49% (120/1264 chunks avaliados). Te

In [11]:
# CÉLULA 7 - Definição das funções buscar_textos_similares (RETRIEVAL) e gerar_resposta_com_contexto (AUGMENTED GENERATION)

import traceback # Import traceback para uso nos blocos except

# Função buscar_textos_similares 
def buscar_textos_similares(query_text, q_client, collection_name, gemini_model_embedding, top_k=3):
    if not q_client or not GOOGLE_AI_API_KEY: # Verifica se o cliente Qdrant e a chave da API (para config do genai) existem
        print("ERRO: Cliente Qdrant ou configuração da API do Google não disponíveis para busca.")
        return []
    try:
        query_embedding_result = genai.embed_content(
            model=gemini_model_embedding, 
            content=query_text,
            task_type="RETRIEVAL_QUERY"
        )
        query_vector = query_embedding_result['embedding']
        
        # Lembre-se do DeprecationWarning que aparecerá.
        search_results = q_client.search(
            collection_name=collection_name,
            query_vector=query_vector,
            limit=top_k,
            with_payload=True
        )
        
        resultados_formatados = []
        for hit in search_results:
            payload = hit.payload if hit.payload else {}
            texto = payload.get("texto_original", payload.get("texto", "Payload sem texto"))
            pagina = payload.get("pagina_pdf", payload.get("pagina", "N/A"))
            arquivo_origem = payload.get("arquivo_origem", "Desconhecido") # Extrai o nome do arquivo
            score = hit.score
            
            resultados_formatados.append({
                "id": hit.id, 
                "score": score, 
                "texto": texto, 
                "pagina": pagina,
                "arquivo_origem": arquivo_origem # Adiciona ao resultado
            })
        return resultados_formatados
    except Exception as e:
        print(f"Erro durante a busca de similaridade: {e}")
        traceback.print_exc()
        return []

# Função para gerar resposta com base no contexto (RAG)
# MODELO_GEMINI_PARA_GERACAO foi definido na Célula 3
# Ex: MODELO_GEMINI_PARA_GERACAO = "gemini-1.5-flash-latest" 

def gerar_resposta_com_contexto(pergunta, chunks_relevantes, nome_modelo_generativo):
    if not chunks_relevantes:
        # Pode ser útil retornar a mensagem de nenhum contexto encontrado diretamente da função de busca,
        # mas manter aqui também é uma salvaguarda.
        return "Não foram encontrados contextos relevantes nos documentos para responder a esta pergunta."

    contexto_str = "\n\n".join([
        f"Trecho do Arquivo '{chunk.get('arquivo_origem', 'Desconhecido')}', Página {chunk.get('pagina', 'N/A')}:\n\"{chunk.get('texto', '')}\""
        for chunk in chunks_relevantes if chunk.get('texto')
    ])
    
    prompt = f"""Você é um assistente especializado em física. Com base nos seguintes trechos extraídos de documentos, responda à pergunta de forma clara e concisa.
Se a informação não estiver nos trechos fornecidos ou não for suficiente, indique que não pode responder com base no contexto fornecido.

Contextos extraídos:
{contexto_str}

Pergunta do usuário: {pergunta}

Resposta:"""

    print(f"\n--- Prompt enviado para o modelo generativo ({nome_modelo_generativo}) ---")
    # print(prompt) # Descomente para ver o prompt completo no log, pode ser verboso
    print("----------------------------------------------------------")

    try:
        model = genai.GenerativeModel(nome_modelo_generativo) # Usa a variável global MODELO_GEMINI_PARA_GERACAO
        response = model.generate_content(prompt)
        # print(f"Resposta recebida do modelo generativo: {response.text[:200]}...") # Log da resposta, já presente na interface_rag_fisica
        return response.text
    except Exception as e:
        print(f"Erro ao gerar resposta com Gemini ({nome_modelo_generativo}): {e}")
        traceback.print_exc()
        return "Desculpe, ocorreu um erro ao tentar gerar a resposta."

print("Funções auxiliares `buscar_textos_similares` e `gerar_resposta_com_contexto` (ajustadas para múltiplos PDFs) definidas.")

Funções auxiliares `buscar_textos_similares` e `gerar_resposta_com_contexto` (ajustadas para múltiplos PDFs) definidas.


In [13]:
# CÉLULA 8 - Interface gráfica com gradio
import gradio as gr 

# Variáveis globais que esta função usará (definidas em células anteriores):
# qdrant_client, GOOGLE_AI_API_KEY, COLLECTION_NAME, GEMINI_EMBEDDING_MODEL, MODELO_GEMINI_PARA_GERACAO

def interface_rag_fisica(pergunta_usuario):
    """
    Função principal que o Gradio chamará.
    Orquestra a busca de chunks e a geração da resposta.
    """
    if not qdrant_client or not GOOGLE_AI_API_KEY:
        return "ERRO: Configuração inicial incompleta (Qdrant ou API Google não disponíveis)."
    if not pergunta_usuario or not pergunta_usuario.strip():
        return "Por favor, digite sua pergunta sobre física."

    print(f"\n[Gradio Interface] Pergunta recebida: '{pergunta_usuario}'")

    # 1. Buscar chunks relevantes
    chunks_encontrados = buscar_textos_similares(
        query_text=pergunta_usuario,
        q_client=qdrant_client,
        collection_name=COLLECTION_NAME, # Definida na Célula 3
        gemini_model_embedding=GEMINI_EMBEDDING_MODEL, # Definida na Célula 3
        top_k=3 # Número de chunks para usar como contexto
    )

    if not chunks_encontrados:
        print("[Gradio Interface] Nenhum chunk relevante encontrado.")
        return "Desculpe, não encontrei informações relevantes em seus documentos para responder a essa pergunta."

    print(f"[Gradio Interface] Chunks encontrados para contexto: {len(chunks_encontrados)}")
    for i, chunk in enumerate(chunks_encontrados):
        print(f"  Chunk {i+1} (Página: {chunk.get('pagina','N/A')} Score: {chunk.get('score',0):.4f}): {str(chunk.get('texto',''))[:100]}...")


    # 2. Gerar resposta usando os chunks como contexto
    resposta_final = gerar_resposta_com_contexto(
        pergunta=pergunta_usuario,
        chunks_relevantes=chunks_encontrados,
        nome_modelo_generativo=MODELO_GEMINI_PARA_GERACAO # Definida na célula anterior
    )
    
    print(f"[Gradio Interface] Resposta final gerada.")
    return resposta_final

# Criar a interface Gradio
iface = gr.Interface(
    fn=interface_rag_fisica,
    inputs=gr.Textbox(lines=3, placeholder="Digite aqui ...", label="🤔 Pergunte ao Universo da Física:"),
    outputs=gr.Markdown(label="Resposta do Assistente de Física"), # Usar Markdown para melhor formatação
    title="Assistente RAG de Física com Gemini e Qdrant",
    description="Faça uma pergunta sobre o conteúdo do seu PDF de física. O sistema buscará informações relevantes e gerará uma resposta.",
    allow_flagging="never",
    theme=gr.themes.Base(
        primary_hue=gr.themes.colors.indigo,
        secondary_hue=gr.themes.colors.pink,
        neutral_hue=gr.themes.colors.slate,
        # MUDANÇA PRINCIPAL AQUI:
        font=[gr.themes.GoogleFont("JetBrains Mono"), "ui-monospace", "monospace"], 
        text_size=gr.themes.sizes.text_md
    ),# Desabilita o botão de "flag"
    examples=[
        ["O que é velocidade média?"],
        ["Explique as leis de Newton."],
        ["Qual a diferença entre massa e peso?"]
    ],
    submit_btn="Enviar",
    clear_btn="Limpar"
)

# Lançar a interface
# Se estiver no Google Colab ou quiser um link público, use iface.launch(share=True)
# Para execução local, iface.launch() é geralmente suficiente.
if 'qdrant_client' in locals() and qdrant_client and 'GOOGLE_AI_API_KEY' in locals() and GOOGLE_AI_API_KEY :
    print("Lançando interface Gradio...")
    iface.launch()
else:
    print("ERRO: Não foi possível lançar a interface Gradio devido a configurações ausentes (Qdrant client ou Google API Key).")

C:\Users\rebec\anaconda3\Lib\site-packages\gradio\interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Lançando interface Gradio...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
